In [1]:
import gensim


In [11]:
#from nltk import download

#download()

In [6]:
from wikipedia import search, page
titles = search('machine learning')
wikipage = page(titles[0])
sentences = [word_tokenize(sent) for sent in sent_tokenize(wikipage.content)]

In [67]:
#from re import split, escape, sub
#from string import punctuation
from nltk import sent_tokenize, word_tokenize

#table = string.maketrans("","")
with open('quaran.txt') as f:
    text = f.read().replace('\n', '')
    
#sentences = split(r' *[\.,\?!][\'"\)\]]* *', text)
#text.translate(table, punctuation)
#text = sub(ur"\p{P}+", "", text)

sentences = [word_tokenize(sent) for sent in sent_tokenize(text)]

In [69]:
from gensim.models import *
min_count = 1
size = 50
window = 10
 
model = Word2Vec(sentences, min_count=min_count, size=size, window=window,  workers=4)

In [70]:
model

['Ai-Ahqaf',
 'recently',
 'funeral',
 'atten-',
 '2:125«-3',
 '720',
 '36-1',
 'chaff',
 'corners',
 '18:1—']

In [3]:
from gensim import corpora

In [4]:
documents = [ "Human machine interface for lab abc computer applications",
              "A survey of user opinion of computer system response time",
              "The EPS user interface management system",
              "System and human system engineering testing of EPS",
              "Relation of user perceived response time to error measurement",
              "The generation of random binary unordered trees",
              "The intersection graph of paths in trees",
              "Graph minors IV Widths of trees and well quasi ordering",
              "Graph minors A survey"]

In [6]:
# remove common words and tokenize
stoplist = set('for a of the and to in'.split())
texts = [[word for word in document.lower().split() if word not in stoplist]
         for document in documents]

In [7]:
# remove words that appear only once
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

texts = [[token for token in text if frequency[token] > 1]for text in texts]

In [8]:
from pprint import pprint  

pprint(texts)

[['human', 'interface', 'computer'],
 ['survey', 'user', 'computer', 'system', 'response', 'time'],
 ['eps', 'user', 'interface', 'system'],
 ['system', 'human', 'system', 'eps'],
 ['user', 'response', 'time'],
 ['trees'],
 ['graph', 'trees'],
 ['graph', 'minors', 'trees'],
 ['graph', 'minors', 'survey']]
